In [1]:
import pickle
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

In [2]:
train_features = pd.read_pickle('../dataset/all_feature.pkl').reset_index(drop=True)
train_features.shape

(1080976, 328)

In [3]:
model_path = '../dataset/lightgbmmodel.pkl'

In [4]:
luck = 123456

In [5]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [6]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
    'max_depth': [2, 3, 4],
#     'learning_rate': [0.15, 0.3],
        'subsample': [0.9],
        'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [44]:
use_columns = [
    x for x in train_features.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel'
    ]
]

In [8]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [9]:
clf_feture = train_features.sample(1000, random_state=luck)

In [54]:
for x in clf_feture.columns:
    if type(clf_feture[x]) != pd.Series:
        print(x, type(clf_feture[x]))

price_dif_hotel <class 'pandas.core.frame.DataFrame'>
price_dif_hotel_rt <class 'pandas.core.frame.DataFrame'>
price_dif_hotel <class 'pandas.core.frame.DataFrame'>
price_dif_hotel_rt <class 'pandas.core.frame.DataFrame'>


In [48]:
clf_feture.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 901048 to 778863
Columns: 285 entries, orderhour to uid__uid_diff_basic_minprice_lastord_75
dtypes: float16(125), float32(58), int16(12), int32(6), int8(84)
memory usage: 607.4 KB


In [18]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 3 candidates, totalling 3 fits
CPU times: user 201 ms, sys: 76.3 ms, total: 277 ms
Wall time: 670 ms


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.4s finished


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'colsample_bytree': [0.9], 'subsample': [0.9], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [38]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 20
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 1
clf.best_params_['colsample_bytree'] = 1
# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 40
# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1

In [39]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=4000,
    **clf.best_params_)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=20, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=4000, nthread=4,
        num_leaves=40, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    train_features[use_columns],
    train_features['orderlabel'],
    test_size=0.33,
    random_state=42)

In [40]:
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=20,
    eval_metric='auc',
    verbose=True, )

[1]	training's auc: 0.892453	valid_1's auc: 0.889166
Train until valid scores didn't improve in 20 rounds.
[2]	training's auc: 0.902205	valid_1's auc: 0.900372
[3]	training's auc: 0.903522	valid_1's auc: 0.901192
[4]	training's auc: 0.904462	valid_1's auc: 0.901868
[5]	training's auc: 0.906127	valid_1's auc: 0.903785
[6]	training's auc: 0.905933	valid_1's auc: 0.903418
[7]	training's auc: 0.906447	valid_1's auc: 0.90385
[8]	training's auc: 0.907066	valid_1's auc: 0.904256
[9]	training's auc: 0.908764	valid_1's auc: 0.905876
[10]	training's auc: 0.909182	valid_1's auc: 0.906123
[11]	training's auc: 0.909879	valid_1's auc: 0.906768
[12]	training's auc: 0.910234	valid_1's auc: 0.907229
[13]	training's auc: 0.910785	valid_1's auc: 0.907453
[14]	training's auc: 0.911292	valid_1's auc: 0.908343
[15]	training's auc: 0.911787	valid_1's auc: 0.908833
[16]	training's auc: 0.912216	valid_1's auc: 0.909171
[17]	training's auc: 0.912526	valid_1's auc: 0.909398
[18]	training's auc: 0.912805	valid_1'

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=20, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=4000, nthread=4,
        num_leaves=40, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)

In [23]:
def get_score(model, X_train, train_features):
    result = model.predict_proba(X_train)[:, 1]
    train_features['prob'] = np.nan
    train_features.loc[X_train.index, 'prob'] = result
    train_x = train_features.loc[X_train.index, ['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [41]:
train_score = get_score(model, X_train, train_features)

0.355152602039


In [37]:
test_score = get_score(model, X_test, train_features)

0.208128493501


In [48]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/01-2242-importance.txt'

In [49]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

ValueError: Shape of passed values is (1, 325), indices imply (1, 322)

In [46]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'learning_rate: 0.1,max_depth: 20,subsample: 1,colsample_bytree: 1,num_leaves: 40'

In [47]:
print(datetime.now().strftime('%d-%H-%M:'),
      'online', '??',
     'test_score', test_score,
     'train_score', train_score,
     'model_train', model.best_score['training']['auc'],
     'model_test', model.best_score['valid_1']['auc'],
     'n', model.best_iteration,
     'params', parms, end='\n', sep='  ')

01-22-41:  online  ??  test_score  0.208128493501  train_score  0.355152602039  model_train  0.945521460169  model_test  0.926384139075  n  124  params  learning_rate: 0.1,max_depth: 20,subsample: 1,colsample_bytree: 1,num_leaves: 40


In [61]:
print(datetime.now().strftime('%d-%H-%M:'),
      'online', '??',
     'test_score', test_score,
     'train_score', train_score,
     'model_train', model.best_score['training']['auc'],
     'model_test', model.best_score['valid_1']['auc'],
     'n', model.best_iteration,
      'shape', train_features.shape,
     'params', parms, end='\n', sep=',', file=open('lgb_result.txt', 'a+'))

In [62]:
pickle.dump(model, open(model_path, 'wb'))